# Kagayaku Database Setup

## 1. Connect to MySQL and Create Database

In [10]:
import mysql.connector
from mysql.connector import errorcode

# --- User Credentials ---
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "7476"
DB_NAME = "MIDA"
SQLTOOLS_CONNECTION_NAME = "Kagayaku"

def create_database(cursor, db_name):
    try:
        cursor.execute(f"CREATE DATABASE {db_name} DEFAULT CHARACTER SET 'utf8'")
        print(f"Database '{db_name}' created successfully.")
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_DB_CREATE_EXISTS:
            print(f"Database '{db_name}' already exists.")
        else:
            print(f"Failed creating database: {err}")
            exit(1)

try:
    # Connect to MySQL server
    cnx = mysql.connector.connect(user=DB_USER, password=DB_PASSWORD, host=DB_HOST)
    cursor = cnx.cursor()
    print("Successfully connected to MySQL server.")
    
    # Create the database
    create_database(cursor, DB_NAME)
    
    # Switch to the new database
    cnx.database = DB_NAME
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print(f"Using database '{DB_NAME}'.")

Successfully connected to MySQL server.
Database 'MIDA' already exists.
Using database 'MIDA'.


## 2. Create Tables

In [11]:
TABLES = {}
TABLES['Table1'] = (
    "CREATE TABLE `Table1` ("
    "  `S_No` int(11) NOT NULL,"
    "  `Import_Date` date,"
    "  `MIDA_NO` varchar(255),"
    "  `Company_Name` varchar(255),"
    "  `Declaration_Reg_No` varchar(255),"
    "  `Kagayaku_Ref_No` varchar(255),"
    "  `HsCode` varchar(255),"
    "  `Item_Name` varchar(255),"
    "  `Balance_Carried_Forward` decimal(10,2),"
    "  `Quantity` float,"
    "  `Balance` decimal(10,2),"
    "  PRIMARY KEY (`S_No`)"
    ") ENGINE=InnoDB")

TABLES['Table2'] = (
    "CREATE TABLE `Table2` ("
    "  `S_No` int(11) NOT NULL,"
    "  `MIDA_NO` varchar(255),"
    "  `Company_Name` varchar(255),"
    "  `HsCode` varchar(255),"
    "  `Item_Name` varchar(255),"
    "  `Approved_Quantity` decimal(10,2),"
    "  `Remaining_Quantity` decimal(10,2),"
    "  `Date_of_exempt` date,"
    "  `Exemption_start_date` date,"
    "  `Exemption_end_date` date,"
    "  PRIMARY KEY (`S_No`)"
    ") ENGINE=InnoDB")

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print(f"Creating table {table_name}: ", end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table Table1: already exists.
Creating table Table2: already exists.


## 3. Flexible Table Creation (for future use)

In [13]:
def generate_flexible_sql_script(table1_name, table2_name, quantity_data_type, metadata_fields):
    script = f"""
USE Kagayaku_db;

CREATE TABLE IF NOT EXISTS {table1_name} (
    S_No INT PRIMARY KEY,
    Import_Date DATE,
    MIDA_NO VARCHAR(255),
    Company_Name VARCHAR(255),
    Declaration_Reg_No VARCHAR(255),
    Kagayaku_Ref_No VARCHAR(255),
    HsCode VARCHAR(255),
    Item_Name VARCHAR(255),
    Balance_Carried_Forward DECIMAL(10, 2),
    Quantity {quantity_data_type},
    Balance DECIMAL(10, 2)
);

CREATE TABLE IF NOT EXISTS {table2_name} (
    S_No INT PRIMARY KEY,
    MIDA_NO VARCHAR(255),
    Company_Name VARCHAR(255),
    HsCode VARCHAR(255),
    Item_Name VARCHAR(255),
    Approved_Quantity DECIMAL(10, 2),
    Remaining_Quantity DECIMAL(10, 2),
"""

    for field_name, field_type in metadata_fields.items():
        script += f"    {field_name} {field_type},\n"

    script = script.rstrip(",\n") + "\n);"
    return script

In [14]:
cursor.close()
cnx.close()